In [1]:
import pandas as pd
import requests
import os
from google.cloud import bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'secret.json'

bq_client = bigquery.Client()

[parties](https://data.cityofnewyork.us/City-Government/ACRIS-Real-Property-Parties/636b-3b5g)

In [3]:

ACRIS=f'https://data.cityofnewyork.us/resource/636b-3b5g.json?$'

In [6]:
acris_count=f'https://data.cityofnewyork.us/resource/636b-3b5g.json?$select=count(1) as cnt'
cnt_req=requests.get(acris_count).json()
nrows=int(cnt_req[0]['cnt'])

In [7]:
nrows

42227126

In [8]:
limit=10**6
offset=0

offs=list(range(0, nrows, limit))

reqs=[f'{ACRIS}limit={limit}&$offset={o}&$order=good_through_date DESC' for o in offs]

In [9]:
table_id='pluto-panel.real_estate.raw_acris_parties'

job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition='WRITE_APPEND'

        )

In [15]:
prog=1
for r in reqs[34:]:
    print(f'reading {prog} / {len(reqs)}: {r}')
    d=requests.get(r).json()
    print('making dataframe')
    d2=pd.DataFrame(d)
    print('loading to bigquery')
    job = bq_client.load_table_from_dataframe(
            d2, table_id, job_config=job_config)
    job.result()
    prog+=1
    
    

reading 1 / 43: https://data.cityofnewyork.us/resource/636b-3b5g.json?$limit=1000000&$offset=34000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 2 / 43: https://data.cityofnewyork.us/resource/636b-3b5g.json?$limit=1000000&$offset=35000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 3 / 43: https://data.cityofnewyork.us/resource/636b-3b5g.json?$limit=1000000&$offset=36000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 4 / 43: https://data.cityofnewyork.us/resource/636b-3b5g.json?$limit=1000000&$offset=37000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 5 / 43: https://data.cityofnewyork.us/resource/636b-3b5g.json?$limit=1000000&$offset=38000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 6 / 43: https://data.cityofnewyork.us/resource/636b-3b5g.json?$limit=1000000&$offset=39000000&$order=good_through_date DESC
making dataframe
loadin

In [16]:
table = bq_client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 42227126 rows and 11 columns to pluto-panel.real_estate.raw_acris_parties
